# 환경 & 라이브러리 & 데이터로드

In [3]:
# --- 기본 imports ---
import os
import json
import numpy as np
import pandas as pd

# 시각화
import matplotlib.pyplot as plt
import seaborn as sns

# profiling
from ydata_profiling import ProfileReport

pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)

print("Ready")


Ready


In [4]:
def load_csv(path: str):
    return pd.read_csv(path)

def load_tsv(path: str):
    return pd.read_csv(path, sep="\t")

def load_json(path: str):
    if os.path.exists(path):
        with open(path, "r") as f:
            return json.load(f)
    return None

def load_all_data(base_path: str):
    data = {}
    data["ratings"] = load_csv(os.path.join(base_path, "train_ratings.csv"))
    data["titles"] = load_tsv(os.path.join(base_path, "titles.tsv"))
    data["years"] = load_tsv(os.path.join(base_path, "years.tsv"))
    data["genres"] = load_tsv(os.path.join(base_path, "genres.tsv"))
    data["directors"] = load_tsv(os.path.join(base_path, "directors.tsv"))
    data["writers"] = load_tsv(os.path.join(base_path, "writers.tsv"))
    data["item2attributes"] = load_json(os.path.join(base_path, "Ml_item2attributes.json"))
    return data

DATA_PATH = "/data/ephemeral/home/data/raw/train"
data = load_all_data(DATA_PATH)

ratings = data["ratings"].copy()
ratings["datetime"] = pd.to_datetime(ratings["time"], unit="s")

ratings.head(), ratings.shape


(   user  item        time            datetime
 0    11  4643  1230782529 2009-01-01 04:02:09
 1    11   170  1230782534 2009-01-01 04:02:14
 2    11   531  1230782539 2009-01-01 04:02:19
 3    11   616  1230782542 2009-01-01 04:02:22
 4    11  2140  1230782563 2009-01-01 04:02:43,
 (5154471, 4))

# item-level movie_df 생성

In [5]:
# item별 review count
item_review_count = ratings.groupby("item")["user"].count().rename("review_count")

# one-to-many → list
genres_per_item = data["genres"].groupby("item")["genre"].apply(list).rename("genres")
directors_per_item = data["directors"].groupby("item")["director"].apply(list).rename("directors")
writers_per_item = data["writers"].groupby("item")["writer"].apply(list).rename("writers")

# 대표값
year_per_item = data["years"].groupby("item")["year"].first().rename("year")
title_per_item = data["titles"].groupby("item")["title"].first().rename("title")

movie_df = pd.DataFrame({"item": item_review_count.index})
movie_df["review_count"] = item_review_count.values

movie_df = movie_df.merge(genres_per_item, on="item", how="left")
movie_df = movie_df.merge(directors_per_item, on="item", how="left")
movie_df = movie_df.merge(writers_per_item, on="item", how="left")
movie_df = movie_df.merge(year_per_item, on="item", how="left")
movie_df = movie_df.merge(title_per_item, on="item", how="left")

movie_df.head()


,item,review_count,genres,directors,writers,year,title
0,1,12217,"[Adventure, Animation, Children, Comedy, Fantasy]",[nm0005124],"[nm0004056, nm0005124, nm0169505, nm0230032, n...",1995.0,Toy Story (1995)
1,2,3364,"[Adventure, Children, Fantasy]",[nm0002653],"[nm0378144, nm0852430, nm0885575]",1995.0,Jumanji (1995)
2,3,734,"[Comedy, Romance]",[nm0222043],[nm0425756],1995.0,Grumpier Old Men (1995)
3,4,43,"[Comedy, Drama, Romance]",[nm0001845],[nm0060103],1995.0,Waiting to Exhale (1995)
4,5,590,[Comedy],[nm0796124],"[nm0329304, nm0352443, nm0583600, nm0796124]",1995.0,Father of the Bride Part II (1995)


## 3️⃣ datetime 기반 파생 피처 (🔥 요청한 부분)

### 3-1️⃣ item 시작 / 종료 시점

In [6]:
item_first_dt = ratings.groupby("item")["datetime"].min().rename("start_datetime")
item_last_dt  = ratings.groupby("item")["datetime"].max().rename("end_datetime")

movie_df = movie_df.merge(item_first_dt, on="item", how="left")
movie_df = movie_df.merge(item_last_dt, on="item", how="left")

movie_df["activity_span"] = movie_df["end_datetime"] - movie_df["start_datetime"]
movie_df["activity_days"] = movie_df["activity_span"].dt.days

movie_df[[
    "start_datetime", "end_datetime", "activity_days"
]].head()


,start_datetime,end_datetime,activity_days
0,2005-04-11 12:41:27,2015-03-30 19:57:57,3640
1,2005-04-13 07:34:58,2015-03-30 12:31:07,3638
2,2005-04-11 17:55:49,2015-03-26 18:25:06,3636
3,2005-05-09 10:54:42,2015-03-22 21:45:09,3604
4,2005-04-13 16:16:13,2015-03-22 21:52:02,3630


### 3-2️⃣ start / end datetime 분해 (year, month, weekday, hour)

In [ ]:
# start datetime 분해
movie_df["start_year"] = movie_df["start_datetime"].dt.year
movie_df["start_month"] = movie_df["start_datetime"].dt.month
movie_df["start_hour"] = movie_df["start_datetime"].dt.hour
movie_df["start_is_weekend"] = (movie_df["start_datetime"].dt.weekday >= 5).astype(int)

# end datetime 분해
movie_df["end_year"] = movie_df["end_datetime"].dt.year
movie_df["end_month"] = movie_df["end_datetime"].dt.month
movie_df["end_hour"] = movie_df["end_datetime"].dt.hour
movie_df["end_is_weekend"] = (movie_df["end_datetime"].dt.weekday >= 5).astype(int)

movie_df[[
    "start_year", "start_month", "start_is_weekend",
    "end_year", "end_month", "end_is_weekend"
]].head()


,start_year,start_month,start_weekday,end_year,end_month,end_weekday
0,2005,4,0,2015,3,0
1,2005,4,2,2015,3,0
2,2005,4,0,2015,3,3
3,2005,5,0,2015,3,6
4,2005,4,2,2015,3,6


## 4️⃣ user 구성 피처

In [8]:
user_activity_count = ratings.groupby("user")["item"].count()

item_users = ratings.groupby("item")["user"].apply(list)

def summarize_users(user_list):
    users = pd.Series(user_list)
    return pd.Series({
        "item_user_median_activity": user_activity_count.loc[users].median(),
        "item_user_heavy_ratio": (
            user_activity_count.loc[users] >
            user_activity_count.quantile(0.9)
        ).mean()
    })

item_user_summary = item_users.apply(summarize_users)
movie_df = movie_df.merge(item_user_summary, on="item", how="left")


## 5️⃣ 콘텐츠 multiplicity 피처

In [9]:
def safe_len(x):
    return len(x) if isinstance(x, list) else 0

movie_df["n_genres"] = movie_df["genres"].apply(safe_len)
movie_df["n_writers"] = movie_df["writers"].apply(safe_len)
movie_df["n_directors"] = movie_df["directors"].apply(safe_len)

movie_df[["n_genres", "n_writers", "n_directors"]].describe()


,n_genres,n_writers,n_directors
count,6807.000000,6807.000000,6807.000000
mean,2.340679,1.660937,0.867489
std,1.114049,1.597339,0.566536
min,1.000000,0.000000,0.000000
25%,1.000000,1.000000,1.000000
50%,2.000000,1.000000,1.000000
75%,3.000000,2.000000,1.000000
max,10.000000,24.000000,14.000000


## 6️⃣ 장르 조합 분석

In [10]:
movie_df["genre_combo"] = movie_df["genres"].apply(
    lambda x: tuple(sorted(x)) if isinstance(x, list) and len(x) > 0 else ("UNKNOWN",)
)

genre_combo_stats = (
    movie_df
    .groupby("genre_combo")
    .agg(
        n_items=("item", "count"),
        mean_review_count=("review_count", "mean"),
        median_review_count=("review_count", "median"),
    )
    .reset_index()
)


# Profiling (최종 요약)

In [11]:
profile = ProfileReport(
    movie_df,
    title="Movie Item-level EDA with Datetime Features",
    explorative=True,
    correlations={
        "pearson": {"calculate": True},
        "spearman": {"calculate": True},
    }
)

profile.to_file("movie_df_profile_with_datetime.html")
print("📄 profiling report 생성 완료")


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 25/25 [00:00<00:00, 52.98it/s]
/data/ephemeral/home/py310/lib/python3.10/site-packages/ydata_profiling/model/pandas/discretize_pandas.py:52: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[9 9 9 ... 0 0 0]' has dtype incompatible with timedelta64[ns], please explicitly cast to a compatible dtype first.
  discretized_df.loc[:, column] = self._discretize_column(
/data/ephemeral/home/py310/lib/python3.10/site-packages/ydata_profiling/model/pandas/discretize_pandas.py:52: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0 0 0 ... 8 8 8]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  discretized_df.loc[:, column] = self._discretize_column(
/data/ephemeral/home/py310/lib/python3.10/site-packages/ydata_profiling/model/pandas/discretize_pandas.py:52: FutureWarning: Setting an item of incompatible

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

📄 profiling report 생성 완료
